<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/unluckid/model/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# 설정

In [15]:
import gdown
import numpy as np
import cv2
import numpy as np
import os
import shutil
file_id = '1jkAAF7VwsXaBh2DgTkP5w9icS1FfutOR'
file_id1 = '1oPazCje0mkB101pzkY1ySCW6g750-KRK'
download_url = f'https://drive.google.com/uc?id={file_id}'
download_url1 = f'https://drive.google.com/uc?id={file_id1}'

# 다운로드 시작

gdown.download(download_url1, '/content/vgg16.keras', quiet=False)
gdown.download(download_url, '/content/inceptionV3.keras', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1oPazCje0mkB101pzkY1ySCW6g750-KRK
From (redirected): https://drive.google.com/uc?id=1oPazCje0mkB101pzkY1ySCW6g750-KRK&confirm=t&uuid=06a98586-eff3-4cb7-ba44-472da8ed0002
To: /content/vgg16.keras
100%|██████████| 301M/301M [00:03<00:00, 80.6MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1jkAAF7VwsXaBh2DgTkP5w9icS1FfutOR
From (redirected): https://drive.google.com/uc?id=1jkAAF7VwsXaBh2DgTkP5w9icS1FfutOR&confirm=t&uuid=8893eefa-5502-44b1-82a1-f255489fae4c
To: /content/inceptionV3.keras
100%|██████████| 403M/403M [00:03<00:00, 117MB/s]


'/content/inceptionV3.keras'

In [19]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import matplotlib.pyplot as plt
from sklearn.ensemble import VotingClassifier,BaggingClassifier
from tensorflow.keras.applications import VGG16, Xception , InceptionV3
from tensorflow.keras.models import load_model

In [18]:
train_dir = "train/nature"
val_dir = "val/nature"
# 데이터 증강 (학습 데이터에만 적용)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=180,       # 회전 범위 증가
)

# 검증 데이터는 증강하지 않음 (단순히 정규화만 적용)
val_datagen = ImageDataGenerator(rescale=1./255)

# 학습 데이터 생성기
train_generator = train_datagen.flow_from_directory(
    train_dir,  # 학습 데이터 경로
    target_size=(224, 224),  # VGG16의 입력 크기에 맞게 리사이즈
    batch_size=32,
    class_mode='categorical',  # 다중 클래스 분류
    shuffle=True
)

# 검증 데이터 생성기
val_generator = val_datagen.flow_from_directory(
    val_dir,  # 검증 데이터 경로
    target_size=(224, 224),  # VGG16의 입력 크기에 맞게 리사이즈
    batch_size=32,
    class_mode='categorical'  # 다중 클래스 분류
)

# tf.data.Dataset으로 변환
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 10), dtype=tf.float32)
    )
)

val_dataset = tf.data.Dataset.from_generator(
    lambda: val_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 10), dtype=tf.float32)
    )
)
train_dataset = train_dataset.repeat()  # 학습 데이터 무한 반복
val_dataset = val_dataset.repeat()      # 검증 데이터 무한 반복


Found 4872 images belonging to 10 classes.
Found 852 images belonging to 10 classes.


In [20]:
model_path1 = '/content/vgg16.keras'
model_path2 = '/content/inceptionV3.keras'
# 모델 불러오기
model1 = load_model(model_path1)
model2 = load_model(model_path2)
# 모델 구조 확인
model1.summary()
model2.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 10 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 177,745,312 (678.04 MB)

 Trainable params: 51,385,866 (196.02 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

 Optimizer params: 102,771,734 (392.04 MB)

In [ ]:
def addlayer(model):
# 모델에 Flatten 레이어 추가
  x = layers.Flatten()(model.output)
  # Dense 레이어 추가 (출력 클래스 수에 맞게)
  x = layers.Dense(512, activation='relu')(x)
  x = layers.Dropout(0.6)(x)  # Dropout 추가
  x = layers.Dense(10, activation='softmax')(x)
  made_model = models.Model(inputs=model.input, outputs=x)
  return made_model

In [ ]:
vgg16 = addlayer(vgg16_base)
inceptionv3 = addlayer(inceptionv3_base)
xception = addlayer(xception_base)

for layer in vgg16_base.layers:
    layer.trainable = False
for layer in inceptionv3_base.layers:
    layer.trainable = False
for layer in xception_base.layers:
    layer.trainable = False
# 모델 컴파일
vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
inceptionv3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
xception.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# 모델 요약 정보 출력
#vgg16.summary()
#ResNet50.summary()
#inceptionv3.summary()
#xception.summary()

In [ ]:
# steps_per_epoch, validation_steps 설정
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = val_generator.samples // val_generator.batch_size
# 모델 학습
def hist(model,train_dataset,steps_per_epoch,val_dataset,validation_steps):
  history = model.fit(
      train_dataset,  # tf.data.Dataset 사용
      steps_per_epoch=steps_per_epoch,  # 배치 크기에 맞춰 학습 스텝 설정
      epochs=50,  # 에포크 수
      validation_data=val_dataset,  # 검증 데이터
      validation_steps=validation_steps  # 검증 배치 크기 맞추기
  )
  return history
#vgg16_h =  hist(vgg16,train_dataset,steps_per_epoch,val_dataset,validation_steps)
inceptionv3 =  hist(vgg16,train_dataset,steps_per_epoch,val_dataset,validation_steps)
inceptionv3.save('inceptionv3.keras')
xception_h =  hist(vgg16,train_dataset,steps_per_epoch,val_dataset,validation_steps)
xception.save('xception.keras')
# 모델 저장
#vgg16.save('vgg16.keras')


Epoch 1/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 3283s 22s/step - accuracy: 0.4416 - loss: 2.0022 - val_accuracy: 0.7392 - val_loss: 0.8275
Epoch 2/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 3188s 21s/step - accuracy: 0.6282 - loss: 0.8885 - val_accuracy: 0.8183 - val_loss: 0.5758
Epoch 3/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 3210s 21s/step - accuracy: 0.6981 - loss: 0.7171 - val_accuracy: 0.8598 - val_loss: 0.5255
Epoch 4/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 3172s 21s/step - accuracy: 0.7135 - loss: 0.6829 - val_accuracy: 0.8634 - val_loss: 0.4298
Epoch 5/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 3177s 21s/step - accuracy: 0.7487 - loss: 0.6093 - val_accuracy: 0.8829 - val_loss: 0.3687
Epoch 6/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 3176s 21s/step - accuracy: 0.7585 - loss: 0.5762 - val_accuracy: 0.8854 - val_loss: 0.3352
Epoch 7/20
152/152 ━━━━━━━━━━━━━━━━━━━━ 3173s 21s/step - accuracy: 0.7732 - loss: 0.5490 - val_accuracy: 0.9256 - val_loss: 0.3005
Epoch 8/20
 61/152 ━━━━━━━━━━━━━━━━━━━━ 27:19 18s/step - accuracy: 0.7960 - loss: 0

In [ ]:
import matplotlib.pyplot as plt

history_list = [vgg16_h,  inceptionv3_h, xception_h]
model_names = ['VGG16',  'InceptionV3', 'Xception']

# 각 모델에 대해 학습 및 검증 정확도와 손실을 그리는 반복문
for i, history in enumerate(history_list):
    plt.figure(figsize=(12, 5))

    # 정확도 그래프
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{model_names[i]} - Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # 손실 그래프
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{model_names[i]} - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.ensemble import VotingClassifier
named_estimators = [
    ("inceptionv3", inceptionv3),
    ("xception_h",  xception_h),
    ("vgg16", vgg16),
]
voting_clf = VotingClassifier(named_estimators)